# Random Numbers and some Statistical Math
- Explain how random numbers are generated in python
- Show some distributions, create some random data calculate some estimators for e.g. the mean
- Do e.g. buffon's needle